### Anna Urbala
# Implementacja NEAT

Z racji tego, że neat jest skomplikowanym podejściem, jego implementacja rozbita jest na dwa tygodnie. W ramach pierwszego deadline należy przesłać niekoniecznie kompletny i działający kod, który będzie dowodem rozpoczęcia pracy nad rozwiązaniem. Dopiero kod w kolejnym tygodniu musi w pełni działać.

W tym tygodniu trzeba zaimplementować: kodowanie, w szczególności generowanie fenotypu z genotypu, wyliczanie funkcji dopasowania i część operatora mutacji polegająca na dodaniu neuronu i zmianie wagi.

W ramach testu przeprowadzić próbne uczenie (bez krzyżowania) na zbiorze easy z laboratorium o sieciach neuronowych.


https://github.com/SirBob01/NEAT-Python/blob/master/neat/neat.py

https://github.com/goktug97/NEAT/blob/master/neat/neat.py

http://nn.cs.utexas.edu/downloads/papers/stanley.cec02.pdf

In [7]:
# todo
# - mutacja biasu
# - mutacja funkcji aktywacji
from enum import Enum, auto
import random
from copy import deepcopy
import math
import operator


def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def relu(x):
    return max(x, 0)

def tanh(x):
    return math.tanh(x)

functions = [sigmoid, relu, tanh]

class NodeType(Enum):
    INPUT = auto()
    HIDDEN = auto()
    OUTPUT = auto()

class ConnectionGene:
    def __init__(self, inNode: int, outNode: int, weight=1.0, expressed=True, innovation=0):
        self.inNode = inNode
        self.outNode = outNode
        self.weight = weight
        self.expressed = expressed
        self.innovation = innovation
    
    def disable(self):
        self.expressed = False

class NodeGene:
    def __init__(self, type_: NodeType, id_: int, bias: float, fun):
        self.type = type_
        self.id = id_
        self.bias = bias
        self.fun = fun

class Counter:
    def __init__(self):
        self.innovation = 0
        
    def getInnovation(self):
        self.innovation += 1
        return self.innovation
    
class Genome:
    PROB_PERTURBING = 0.9
    
    def __init__(self):
        self.connections = {}
        self.nodes = {}
    
    def addNodeGene(self, gene: NodeGene):
        self.nodes[gene.id] = gene
    
    def addConnectionGene(self, gene: ConnectionGene):
        self.connections[gene.innovation] = gene
        
    def mutation(self):
        for conn in self.connections.values():
            if random.uniform(0, 1.0) < self.PROB_PERTURBING:
                conn.weight *= random.uniform(-2.0, 2.0)
            else:
                conn.weight = random.uniform(-2.0, 2.0)
            
    
    def addConnectionMutation(self, innovation: Counter, maxAttempts):
        tries = 0
        success = False
        while tries < maxAttempts and success == False:
            tries += 1
            node1 = self.nodes[random.randrange(len(self.nodes))]
            node2 = self.nodes[random.randrange(len(self.nodes))]

            rev = False
            if node1.type.value > node2.type.value:
                rev = True
                node1,node2 = node2,node1
            
            connectionImpossible = False
            if (node1.type == NodeType.INPUT and node2.type == NodeType.INPUT) \
            or (node1.type == NodeType.OUTPUT and node2.type == NodeType.OUTPUT):
                connectionImpossible = True

            connectionExists = False
            for conn in self.connections.values():
                if (conn.inNode == node1.id and conn.outNode == node2.id) \
                or (conn.inNode == node2.id and conn.outNode == node1.id):
                    connectionExists = True
                    break
            
            if connectionExists or connectionImpossible:
                continue

            newConn = ConnectionGene(node1.id, node2.id, weight=random.uniform(-1.0, 1.0), innovation=innovation.getInnovation())
            self.addConnectionGene(newConn)
            success = True
    
    def addNodeMutation(self, connectionInnovation: Counter, nodeInnovation: Counter):
        conn = self.connections[random.randrange(len(self.connections))]
        
        inNode = self.nodes[conn.inNode]
        outNode = self.nodes[conn.outNode]
        
        conn.disable()
        
        newNode = NodeGene(NodeType.HIDDEN, nodeInnovation.getInnovation(), random.uniform(-1, 1), random.choice(functions))
        newConn1 = ConnectionGene(inNode.id, newNode.id, innovation=connectionInnovation.getInnovation())
        newConn2 = ConnectionGene(newNode.id, outNode.id, weight=conn.weight, innovation=connectionInnovation.getInnovation())
        
        self.addNodeGene(newNode)
        self.addConnectionGene(newConn1)
        self.addConnectionGene(newConn2)
    
    
    @staticmethod
    def crossover(parent1, parent2):
        child = Genome()
        for parent1Node in parent1.nodes.values():
            child.nodes.append(deepcopy(parent1Node))
        
        for parent1Conn in paren1.connections.values():
            childConnGene = deepcopy(parent1Conn)
            if parent1Conn.innovation in parent2.connections and random.choice([True, False]):
                childConnGene = deepcopy(parent2.connections[parent1Conn.innovation])
            child.addConnectionGene(childConnGene)
            
        return child
    
    @staticmethod
    def compatibilityDistance(genome1, genome2, c1: int, c2: int, c3: int):
        excessGenes = Genome.countExcessGenes(genome1, genome2)
        disjointGenes = Genome.countDisjointGenes(genome1, genome2)
        avgWeightDiff = Genome.averageWeightDiff(genome1, genome2)
        return excessGenes * c1 + disjointGenes * c2 + avgWeightDiff * c3
    
    @staticmethod
    def countMatchingGenes(genome1, genome2):
        matchingGenes = 0
        
        highestInnovation1 = max(genome1.nodes.keys())
        highestInnovation2 = max(genome2.nodes.keys())
        indices = max(highestInnovation1, highestInnovation2)
        
        for i in range(indices + 1):
            if i in genome1.nodes and i in genome2.nodes:
                matchingGenes += 1
        
        highestInnovation1 = max(genome1.connections.keys())
        highestInnovation2 = max(genome2.connections.keys())
        indices = max(highestInnovation1, highestInnovation2)
        
        for i in range(indices + 1):
            if i in genome1.connections and i in genome2.connections:
                matchingGenes += 1
        
        return matchingGenes
    
    @staticmethod
    def countDisjointGenes(genome1, genome2):
        disjointGenes = 0
        
        highestInnovation1 = max(genome1.nodes.keys())
        highestInnovation2 = max(genome2.nodes.keys())
        indices = min(highestInnovation1, highestInnovation2)
        
        for i in range(indices + 1):
            if (i not in genome1.nodes and i in genome2.nodes) \
            or (i not in genome2.nodes and i in genome1.nodes):
                disjointGenes += 1
        
        highestInnovation1 = max(genome1.connections.keys())
        highestInnovation2 = max(genome2.connections.keys())
        indices = min(highestInnovation1, highestInnovation2)
        
        for i in range(indices + 1):
            if (i not in genome1.connections and i in genome2.connections) \
            or (i not in genome2.connections and i in genome1.connections):
                disjointGenes += 1
        
        return disjointGenes
    
    @staticmethod
    def countExcessGenes(genome1, genome2):
        excessGenes = 0
        
        highestInnovation1 = max(genome1.nodes.keys())
        highestInnovation2 = max(genome2.nodes.keys())
        indices1 = min(highestInnovation1, highestInnovation2)
        indices2 = max(highestInnovation1, highestInnovation2)
        
        for i in range(indices1+1, indices2+1):
            if (i not in genome1.nodes and i in genome2.nodes) \
            or (i not in genome2.nodes and i in genome1.nodes):
                excessGenes += 1
        
        highestInnovation1 = max(genome1.connections.keys())
        highestInnovation2 = max(genome2.connections.keys())
        indices1 = min(highestInnovation1, highestInnovation2)
        indices2 = max(highestInnovation1, highestInnovation2)
        
        for i in range(indices1+1, indices2+1):
            if (i not in genome1.connections and i in genome2.connections) \
            or (i not in genome2.connections and i in genome1.connections):
                excessGenes += 1
        
        return excessGenes
    
    @staticmethod
    def averageWeightDiff(genome1, genome2):
        matchingGenes = 0
        weightDifference = 0.0

        highestInnovation1 = max(genome1.connections.keys())
        highestInnovation2 = max(genome2.connections.keys())
        indices = max(highestInnovation1, highestInnovation2)
        
        for i in range(indices + 1):
            if i in genome1.connections and i in genome2.connections:
                matchingGenes += 1
                weightDifference += abs(genome1.connections[i].weight - genome2.connections[i].weight)
        
        return weightDifference/matchingGenes
    
    def forward(self, inputs):
        """Evaluate inputs and calculate the outputs of the
        neural network via the forward propagation algorithm.
        """
        _inputs = [node for self.nodes.values() if node.type == NodeType.INPUT]
        _inputs.sort(key=operator.attrgetter('id'))

        # Set input values
        for i, node in enumerate(_inputs):
            node.output = inputs[i]
        
        # Generate backward-adjacency list 
        _from = {}
        _to = {}
        waiting_inputs = {}
        for node in self.nodes.values():
            _from[node.id] = []
            _to[node.id] = []
            waiting_inputs[node.id] = 0

        for conn in self.connections:
            if conn.expressed:
                _from[conn.outNode].append(conn.inNode)
                _to[conn.inNode].append(conn.outNode)
                waiting_inputs[conn.outNode] += 1
                
        ready_nodes = [node for self.nodes.values if waiting_inputs[node.id] == 0]
        # todo
        while len(ready_nodes):
            node = ready_nodes.pop()
        for j in ordered_nodes:
            ax = 0
            for i in _from[j]:
                ax += self._edges[(i, j)].weight * self._nodes[i].output

            node = self._nodes[j]
            node.output = node.activation(ax + node.bias)
        
        return [self._nodes[n].output for n in range(self._inputs, self._unhidden)]
    

In [6]:
from abc import ABC, abstractmethod

class FitnessGenome:
    def __init__(self, genome: Genome, fitness: float):
        self.genome = genome
        self.fitness = fitness
    
class Species:
    def __init__(self, mascot: Genome):
        self.mascot = mascot
        self.members = [mascot]
        self.fitnessPop = []
        self.totalAdjustedFitness = 0.0
    
    def addAdjustedFitness(self, adjustedFitness: float):
        self.totalAdjustedFitness += adjustedFitness
    
    def reset(self):
        newMascotIndex = random.randrange(len(self.members))
        self.mascot = self.members[newMascotIndex]
        self.members = []
        self.fitnessPop = []
        self.totalAdjustedFitness = 0.0
    
class Evaluator(ABC):
    C1 = 1.0
    C2 = 1.0
    C3 = 0.4
    DT = 10.0
    MUTATION_RATE = 0.8
    ADD_CONNECTION_RATE = 0.1
    ADD_NODE_RATE = 0.1
    CROSSOVER_RATE = 0.75
    
    def __init__(self, populationSize: int, startingGenome: Genome, nodeInnovation: Counter, connectionInnovation: Counter):
        self.speciesMap = {} # genome -> species
        self.scoreMap = {} # genome -> float
        self.species = []
        # self.genomes = []
        self.nextGenGenomes = [None for i in range(populationSize)]
        self.populationSize = populationSize
        self.nodeInnovation = nodeInnovation
        self.connectionInnovation = connectionInnovation
        self.highestScore = 0.0
        self.fittestGenome = None
        self.genomes = [deepcopy(startingGenome) for i in range(populationSize)]
    
    def evaluate():
        for s in self.species:
            s.reset()
        
        self.scoreMap = {}
        self.speciesMap = {}
        self.nextGenGenomes = []
        self.highestScore = 0.0
        self.fittestGenome = None
        
        # place genomes into species
        for g in self.genomes:
            foundSpecies = False
            for s in self.species:
                if Genome.compatibilityDistance(g, s.mascot, C1, C2, C3) < DT:
                    s.members.append(g)
                    self.speciesMap[g] = s
                    foundSpecies = True
                    break
            if not foundSpecies:
                newSpecies = Species(g)
                self.species.append(newSpecies)
                self.speciesMap[g] = newSpecies
        
        # remove unused species
        self.species = [s for s in self.species if len(s.members)]
        
        # evaluate genomes and assign fitness
        for g in self.genomes:
            s = speciesMap[g]
            
            score = self.evaluateGenome(g)
            adjustedScore = score / len(s.members)
            
            s.addAdjustedFitness(adjustedFitness)
            s.fitnessPop(FitnessGenome(g, adjustedScore))
            self.scoreMap[g] = adjustedScore
            
            if score > self.highestScore:
                self.highestScore = score
                self.fittestGenome = g
        
        # put best genomes from each species into next generation
        for s in species:
            s.fitnessPop.sort(key=operator.attrgetter('fitness'))
            fittestInSpecies = s.fitnessPop[-1]
            self.nextGenGenomes.append(fittestInSpecies.genome)
        
        # breed the rest of the genomes
        while len(self.nextGenGenomes) < self.populationSize:
            s = self.getRandomSpeciesBiasedAdjustedFitness()
            child = None
            
            if random.uniform(0, 1.0) < self.CROSSOVER_RATE:
                p1 = self.getRandomGenomeBiasedAdjustedFitness(s)
                p2 = self.getRandomGenomeBiasedAdjustedFitness(s)
                if self.scoreMap[p2] > self.scoreMap[p1]:
                    p1, p2 = p2, p1
                child = Genome.crossover(p1, p2)
            else:
                child = deepcopy(self.getRandomGenomeBiasedAdjustedFitness(s))
            
            if random.uniform(0, 1.0) < self.MUTATION_RATE:
                child.mutation()
            if random.uniform(0, 1.0) < self.ADD_CONNECTION_RATE:
                child.addConnectionMutation(connectionInnovation, 10)
            else if random.uniform(0, 1.0) < self.ADD_NODE_RATE:
                child.addNodeMutation(connectionInnovation, nodeInnovation)
            
            self.nextGenGenomes.append(child)
        
        self.genomes = self.nextGenGenomes
        self.nextGenGenomes = []
    
    def getRandomSpeciesBiasedAdjustedFitness(self):
        completeWeight = sum([s.totalAdjustedFitness for s in self.species])
        
        r = random.uniform(0, completeWeight)
        countWeight = 0.0
        for s in self.species:
            countWeight += s.totalAdjustedFitness
            if countWeight >= r:
                return s
        
        raise Exception("Couldn't find a species... " + len(self.species))
       
    def getRandomGenomeBiasedAdjustedFitness(self, selectFrom: Species):
        completeWeight = sum([fg.fitness for fg in selectFrom.fitnessPop])
        
        r = random.uniform(0, completeWeight)
        countWeight = 0.0
        for fg in selectFrom.fitnessPop:
            countWeight += fg.fitness
            if countWeight >= r:
                return fg.genome
        
        raise Exception("Couldn't find a genome... " + len(selectFrom.fitnessPop))
    
    @abstractmethod
    def evaluateGenome(genome: Genome):
        pass
    
    
        

In [30]:
a = {1: "4", 4: "3"}
6 in a

False

In [32]:
abs(-4)

4